In this file we present the code used for searching the fields of Urban Dictionary words regarding the date they were posted (and other info like thumbs up).

In [4]:
import sys
import pandas as pd
import json
if sys.version < '3':
    from urllib2 import urlopen
    from urllib import quote as urlquote
else:
    from urllib.request import urlopen
    from urllib.parse import quote as urlquote
    
def _get_urban_json(url):
    f = urlopen(url)
    data = json.loads(f.read().decode('utf-8'))
    f.close()
    return data  
#UD_DEFID_URL = 'https://api.urbandictionary.com/v0/define?defid='
#UD_DEFINE_URL = 'https://api.urbandictionary.com/v0/define?term='
#UD_RANDOM_URL = 'https://api.urbandictionary.com/v0/random'


In [8]:
data  = pd.read_csv("WordsToProbe.csv")
#Let's make sure that the words are in the 'URL' form
data['word_id_start'] = data['word_id']
data = data[[ 'word_id','final_countries','word_id_start']]
data['word_id'] = data["word_id"].str.replace(" ", "-")
words = list(data['word_id'].drop_duplicates())
dataframe1 = pd.DataFrame({})


We store in dataframe1 all the words and their fields

In [10]:
for i,word_id in enumerate(words):
    try:
        a =_get_urban_json("https://api.urbandictionary.com/v0/define?term='{}'".format(urlquote(word_id)))
        df_urban  = pd.DataFrame.from_dict(a['list'], orient='columns')
        df_urban['word_id'] = word_id
        dataframe1 = pd.concat([dataframe1,df_urban])
        if( i % 100 == 0):
            print(i)
    except urllib.error.HTTPError as e:
        if e.code == 500:
            print("Exception with word {}".format(word_id))
    except TypeError:
        print("Exception type error with word {}".format(word_id))

            


0


as we see, each word is retrieved in all its different submissions. We now go to R to filter out equal words, that is equal with the same word_id

In [11]:
dataframe1  #We show only a sample here

,definition,permalink,thumbs_up,sound_urls,author,word,defid,current_vote,written_on,example,thumbs_down,word_id
0,Also known as “brother”. Is [Alabamian] slang ...,http://quaggy.urbanup.com/15691200,3,[],ItsKdyn,Quaggy,15691200,,2021-01-26T07:46:46.231Z,"[Oh my God], Quaggy! [What are you doing]?",0,Quaggy
1,[soft] and [flabby]; [pillowy],http://quaggy.urbanup.com/297811,10,[],Kae,quaggy,297811,,2003-10-17T16:13:54.000Z,[Hey]! You're [not fat]! You're quaggy!\r\n,11,Quaggy
2,having [a fat] [sook] over [nothing],http://quaggy.urbanup.com/14561985,2,[],gabetndapussyslayer,quaggy,14561985,,2019-12-20T04:53:56.483Z,[oh] have a quaggy you [pussy],2,Quaggy
3,[soft] and [flabby]; [pillowy],http://quaggy.urbanup.com/297817,6,[],Kaela,quaggy,297817,,2003-10-17T16:18:27.000Z,You're [really] quaggy! [You know that]?,12,Quaggy
0,While driving: constantly [changing lanes] and...,http://weaving.urbanup.com/7836595,9,[],Photon_Man62,weaving,7836595,,2014-07-15T17:45:34.301Z,- [Whoa whoa whoa] man! Cut that out! Why are ...,0,Weaved
...,...,...,...,...,...,...,...,...,...,...,...,...
1,a South East-Asian-based slang to [indicate] s...,http://zety.urbanup.com/11086853,14,[],buritbusuk,zety,11086853,,2017-02-23T06:52:54.168Z,Despite sleeping with four random guys that we...,5,zety
2,"A girl who are kind , sometimes [childish] but...",http://zety.urbanup.com/12330805,6,[],Fruity poppin,Zety,12330805,,2017-12-26T18:09:53.273Z,"Hi [zety] ! You look cute today, [thank you] f...",2,zety
0,In [Belgium] they don't say Harry Potter the s...,http://bagety.urbanup.com/13598273,5,[],BAGETY,Bagety,13598273,,2019-02-08T12:02:18.690Z,I am [gonna] [watch] [bagety].,0,Bageti
1,[Asian slang] term for [the food] [baguette].\...,http://bageti.urbanup.com/3923200,7,[],ugi1ugi,Bageti,3923200,,2009-04-28T06:34:13.000Z,[Chef]: Hey have cut that [bageti] yet\r\nCook...,1,Bageti
